# Tweet Scraping
The purpose of this notebook is to collect all of the necessary data using the Twitter API.

Get environment variables from the `.env` file.

In [13]:
from dotenv import load_dotenv
import os

load_dotenv()
TWITTER_API_KEY = os.getenv('TWITTER_API_KEY')
TWITTER_API_SECRET = os.getenv('TWITTER_API_SECRET')
TWITTER_BEARER_TOKEN = os.getenv('TWITTER_BEARER_TOKEN')

Authenticate with a bearer token.

In [14]:
import tweepy

auth = tweepy.OAuth2BearerHandler(TWITTER_BEARER_TOKEN)
api = tweepy.API(auth)

Print out my username and followers to make sure everything is working.

In [15]:
user = api.get_user(screen_name='ishan0102')
print(user.name)
print(user.followers_count)

ishan
227


In [16]:
import csv

# update these for the tweet you want to process replies to 'name' = the account username and you can find the tweet id within the tweet URL
name = 'elonmusk'
tweet_id = '1517234027970899982'

replies=[]
for tweet in tweepy.Cursor(api.search_tweets,q='to:'+name, result_type='recent').items(1000):
    if hasattr(tweet, 'in_reply_to_status_id_str'):
        if (tweet.in_reply_to_status_id_str==tweet_id):
            replies.append(tweet)

with open('replies_clean.csv', 'w') as f:
    csv_writer = csv.DictWriter(f, fieldnames=('user', 'text'))
    csv_writer.writeheader()
    for tweet in replies:
        row = {'user': tweet.user.screen_name, 'text': tweet.text.replace('\n', ' ')}
        csv_writer.writerow(row)